# Fine-tuning performance of multilingual models 

In [ ]:
import torch, os
from google.colab import drive

drive.mount("/content/drive")
if os.getcwd() != "/content/drive/My Drive/akademi/Packt NLP with Transformers/CH09":
    os.chdir("drive/MyDrive/akademi/Packt NLP with Transformers/CH09")

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install transformers datasets

## Loading Pre-Trained Model

In [ ]:
from torch import cuda

device = "cuda" if cuda.is_available() else "cpu"
device

### Obtaining and Preparing downstream task data

In [ ]:
if "TTC4900.csv" not in os.listdir():
    !wget  https://raw.githubusercontent.com/savasy/TurkishTextClassification/master/TTC4900.csv
else:
    print("Already there !")

In [ ]:
import pandas as pd

data = pd.read_csv("TTC4900.csv")
data = data.sample(frac=1.0, random_state=42)
data.head(5)

In [ ]:
labels = ["teknoloji", "ekonomi", "saglik", "siyaset", "kultur", "spor", "dunya"]
NUM_LABELS = len(labels)
id2label = {i: l for i, l in enumerate(labels)}
label2id = {l: i for i, l in enumerate(labels)}

In [ ]:
label2id

In [ ]:
data["labels"] = data.category.map(lambda x: label2id[x.strip()])

In [ ]:
data.head()

In [ ]:
data.category.value_counts().plot(kind="pie", figsize=(8, 8))

## BERT

In [ ]:
# from transformers import BertTokenizerFast
# tokenizer = BertTokenizerFast.from_pretrained("dbmdz/bert-base-turkish-uncased", max_length=512)
# from transformers import BertForSequenceClassification
# model = BertForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-uncased", num_labels=NUM_LABELS, id2label=id2label, label2id=label2id)

# We will run the fine-tuning computations from the previous chapter again with new pre-traine model

We either instatiate mBERT or XLM-R. Firts we go with mBERT, then we will use XLM-R. Currently we instantiate mBERT and we comment out XLM-R code block as follows:


# mBERT

In [ ]:
from transformers import BertForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-uncased",
    num_labels=NUM_LABELS,
    id2label=id2label,
    label2id=label2id,
)
model.to(device)

## XLM-R

Official Note of XLM-R:
This model was pre-trained on set of 100 languages, as described in the original paper.
It was then fine-tuned on the task of NLI on the concatenated MNLI train set and the XNLI validation and test sets. Finally, it was trained for one additional epoch on only XNLI data where the translations for the premise and hypothesis are shuffled such that the premise and hypothesis for each example come from the same original English example but the premise and hypothesis are of different languages.

In [ ]:
"""
from transformers import AutoTokenizer, XLMRobertaForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base",num_labels=NUM_LABELS, id2label=id2label, label2id=label2id)
model.to(device)
"""

In [ ]:
!nvidia-smi

## Splitting data


In [ ]:
SIZE = data.shape[0]

train_texts = list(data.text[: SIZE // 2])
val_texts = list(data.text[SIZE // 2 : (3 * SIZE) // 4])
test_texts = list(data.text[(3 * SIZE) // 4 :])

train_labels = list(data.labels[: SIZE // 2])
val_labels = list(data.labels[SIZE // 2 : (3 * SIZE) // 4])
test_labels = list(data.labels[(3 * SIZE) // 4 :])

In [ ]:
len(train_texts), len(val_texts), len(test_texts)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
from torch.utils.data import Dataset


class MyDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = MyDataset(train_encodings, train_labels)
val_dataset = MyDataset(val_encodings, val_labels)
test_dataset = MyDataset(test_encodings, test_labels)

## Training with Trainer Class

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="macro"
    )
    acc = accuracy_score(labels, preds)
    return {"Accuracy": acc, "F1": f1, "Precision": precision, "Recall": recall}

In [ ]:
training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir="./TTC4900Model",
    do_train=True,
    do_eval=True,
    #  The number of epochs, defaults to 3.0
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    # Number of steps used for a linear warmup
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy="steps",
    # TensorBoard log directory
    logging_dir="./multi-class-logs",
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="epoch",
    fp16=True,
    load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    # the pre-trained model that will be fine-tuned
    model=model,
    # training arguments that we defined above
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
q = [
    trainer.evaluate(eval_dataset=data)
    for data in [train_dataset, val_dataset, test_dataset]
]
pd.DataFrame(q, index=["train", "val", "test"]).iloc[:, :5]